# OpenAP - Estimation Fuel and Emissions

## Install OpenAP


In [ ]:
%pip install OpenAP
%pip install cartopy

Import packages

In [ ]:
import openap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
flight = pd.read_csv('https://github.com/junzis/tutorial-icrat24-openap/raw/master/data/flight_a320.csv')

In [ ]:
flight.head()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 8), sharex=True) 

ax1.plot(flight.time, flight.altitude, label="altitude")
ax2.plot(flight.time, flight.vertical_rate, label="vertical rate")
ax3.plot(flight.time, flight.groundspeed, label="groundspeed")
ax3.plot(flight.time, flight.true_airspeed, label="true airspeed")
for ax in (ax1, ax2, ax3):
    ax.legend(loc="lower center")
ax3.set_xlabel("time [s]")
plt.show()

In [ ]:
ff = openap.FuelFlow(ac="a320")

fuel_flow_estimate = ff.enroute(
    flight.mass, flight.true_airspeed, flight.altitude, flight.vertical_rate
)


In [ ]:
plt.plot(flight.time, flight.fuel_flow, label="actual")
plt.plot(flight.time, fuel_flow_estimate, label="estimate", alpha=0.7)
plt.legend()
plt.show()

In [ ]:
emi = openap.Emission(ac="a320")

co2 = emi.co2(fuel_flow_estimate)
h2o = emi.h2o(fuel_flow_estimate)
soot = emi.soot(fuel_flow_estimate)

nox = emi.nox(fuel_flow_estimate, flight.true_airspeed, flight.altitude)
co = emi.co(fuel_flow_estimate, flight.true_airspeed, flight.altitude)
hc = emi.hc(fuel_flow_estimate, flight.true_airspeed, flight.altitude)

fig, axes = plt.subplots(3, 2, figsize=(12, 6), sharex=True)

colors = plt.cm.tab10.colors
gases = [co2, nox, h2o, co, soot, hc]
labels = ["CO2", "NOx", "H2O", "CO", "Soot", "HC"]
for i, (gas, ax) in enumerate(zip(gases, axes.flat)):
    ax.plot(flight.time, gas, color=colors[i], label=labels[i])
    ax.legend(loc="upper center")
    ax.tick_params(axis='y', colors=colors[i])
for ax in axes[-1, :]:
    ax.set_xlabel("time [s]")


# Estimating fuel consumption from ADS-B data

In [ ]:
flight = pd.read_csv('https://github.com/junzis/tutorial-icrat24-openap/raw/master/data/hv5472_20240623.csv')

In [ ]:
flight

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig, ax = plt.subplots(1, 1, figsize=(8, 8), subplot_kw={"projection": ccrs.EuroPP()})

ax.add_feature(cfeature.LAND, color="lightgray")
ax.add_feature(cfeature.COASTLINE, lw=0.5, color="darkgray")
ax.add_feature(cfeature.BORDERS, lw=0.5, color="gray")

ax.set_extent(
    [
        flight.longitude.min() - 10,
        flight.longitude.max() + 10,
        flight.latitude.min() - 2,
        flight.latitude.max() + 2,
    ]
)

ax.plot(flight.longitude, flight.latitude, transform=ccrs.Geodetic(), lw=2, color="red")
plt.show()

In [ ]:
import openap.prop

aircraft = openap.prop.aircraft("B738")

ff = openap.FuelFlow(ac="B738")

mass_init = aircraft["mtow"] * 0.9

In [ ]:
mass = mass_init
timestamp = flight.timestamp.iloc[0]

fuel_flow_estimate = []

for i, d in flight.iterrows():
    ffe = ff.enroute(mass=mass, tas=d.groundspeed, alt=d.altitude, vs=d.vertical_rate)
    mass -= ffe * d.d_time
    fuel_flow_estimate.append(ffe)


plt.plot(flight.timestamp, fuel_flow_estimate, label="fuel flow estimate (kg/s)")
plt.legend()
plt.show()